In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from rich import print
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=False, sep=" | ", start_row=None, end_row=None
)

# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=",", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=False, sep=" -> ", start_row=None, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=False, sep=" ", start_row=2, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="si", per_letter=False, sep=" ", start_row=0, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=0, end_row=None
# )

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: bceadfg ebcdaf gdaecf fgcaeb fdbea fcdea cbda begdf afb ba | acfbeg cbfgea gebacf
ab | Last line: ecfba egf dgbeaf dbgcfea gf egfcb egabcd cfgd gefcdb egdcb | gbfec dcfg fg 
fbgce

------

Convert to:

First line: ['bceadfg ebcdaf gdaecf fgcaeb fdbea fcdea cbda begdf afb ba', 'acfbeg cbfgea 
gebacf ab']

Last line: ['ecfba egf dgbeaf dbgcfea gf egfcb egabcd cfgd gefcdb egdcb', 'gbfec dcfg fg 
fbgce']

------

In [4]:
all_lines_li = []
for l in data:
    all_lines_li.append([*l[0].split(), *l[1].split()])

letter_to_digit_map = {
    "abcefg": 0,
    "cf": 1,
    "acdeg": 2,
    "acdfg": 3,
    "bcdf": 4,
    "abdfg": 5,
    "abdefg": 6,
    "acf": 7,
    "abcdefg": 8,
    "abcdfg": 9,
}

decode_li = []

for line in all_lines_li:
    char_map = {c: "" for c in "abcdefg"}
    # Count len
    count_len = np.array([len(l) for l in line[0:10]])
    # Combine all the letters together
    letter_comb = list("".join(line[0:10]))
    # Count letters
    values, counts = np.unique(letter_comb, return_counts=True)
    for value, count in zip(values, counts):
        # If a letter appears 4 times, it must be "e"
        if count == 4:
            char_map[value] = "e"
        elif count == 9:  # must be "f"
            letter_for_f = value
            char_map[letter_for_f] = "f"
            # 2 digits, the other char must be c
            letter_for_c = (set(line[np.argmin(count_len)]) - set(value)).pop()
            char_map[letter_for_c] = "c"
        elif count == 6:  # must be "b"
            letter_for_b = value
            char_map[letter_for_b] = "b"

    for value, count in zip(values, counts):
        # If a letter appears 8 times, one is "c", the other one must be "a"
        if count == 8:
            if value != letter_for_c:
                char_map[value] = "a"

    # 4 digits, they are bcdf, the 4th one must be "d"
    char_map[
        (
            set(np.array(line[0:10])[count_len == 4][0])
            - set([letter_for_b, letter_for_c, letter_for_f])
        ).pop()
    ] = "d"
    # The last one is "g"
    for k in char_map.keys():
        if char_map[k] == "":
            char_map[k] = "g"

    codes = []
    for l in line[10:]:
        temp_li = [char_map[letter] for letter in l]
        temp_li.sort()
        new_l = "".join(temp_li)
        decode_digi = letter_to_digit_map[new_l]
        codes.append(decode_digi)

    decode_li.append(codes)

code_arr = np.array(decode_li)
np.sum(np.isin(code_arr, [1, 4, 7, 8]))
print(f"Answer to Q1: {np.sum(np.isin(code_arr, [1,4,7,8]))}")
print(f"Answer to Q2: {np.sum([int(''.join([str(j) for j in i])) for i in code_arr])}")

Answer to Q1: 392

Answer to Q2: 1004688